# IMDB 영화 리뷰 감성 분석 (기본 신경망)

이 노트북은 Keras에 내장된 IMDB 데이터셋을 사용하여 영화 리뷰가 긍정적인지 부정적인지를 분류하는 신경망을 구축, 훈련, 평가합니다.

**프로세스:**
1. **데이터 로딩**: IMDB 데이터셋을 로드합니다. 리뷰는 전처리되어 단어 시퀀스를 나타내는 정수 배열로 제공됩니다.
2. **데이터 탐색**: 정수 시퀀스를 다시 텍스트로 디코딩하여 원본 리뷰 내용을 확인합니다.
3. **데이터 전처리**: 신경망에 입력할 수 있도록 정수 시퀀스를 원-핫 인코딩(multi-hot encoding) 방식의 벡터로 변환합니다.
4. **모델 구축**: `Dense` 레이어로 구성된 간단한 순차 모델을 정의합니다.
5. **훈련 및 검증**: 훈련 데이터를 사용하여 모델을 학습시키고, 검증 데이터를 사용하여 성능을 모니터링합니다.
6. **결과 시각화 및 평가**: 훈련 과정의 정확도와 손실을 그래프로 시각화하고, 테스트 데이터셋으로 최종 성능을 평가합니다.

## 1. 라이브러리 임포트

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.datasets import imdb
import matplotlib.pyplot as plt

## 2. 데이터 로딩

In [ ]:
# 가장 빈도가 높은 10,000개의 단어만 사용하도록 데이터 로드
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

print(f"훈련 데이터 수: {len(train_data)}")
print(f"테스트 데이터 수: {len(test_data)}")
print("첫 번째 훈련 데이터 (정수 시퀀스):\n", train_data[0])
print("첫 번째 훈련 레이블:", train_labels[0])

## 3. 데이터 탐색 (정수 시퀀스를 텍스트로 변환)

In [ ]:
# 단어와 정수 인덱스를 매핑하는 딕셔너리 로드
word_index = imdb.get_word_index()

# 정수 인덱스와 단어를 매핑하도록 딕셔너리 반전
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_review(text_sequence):
    # 0, 1, 2는 '패딩', '문서 시작', '사전 외 단어'를 위한 특수 인덱스이므로 3을 뺌
    return " ".join([reverse_word_index.get(i - 3, "?") for i in text_sequence])

print("첫 번째 리뷰 디코딩 결과:\n", decode_review(train_data[0]))

## 4. 데이터 전처리 (벡터화)

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    # (len(sequences), dimension) 크기의 0으로 채워진 행렬 생성
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        # sequence의 각 인덱스 위치를 1로 설정
        results[i, sequence] = 1.
    return results

# 훈련 데이터와 테스트 데이터를 벡터로 변환
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

# 레이블을 float32 타입의 벡터로 변환
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

print("벡터화된 첫 번째 훈련 데이터 샘플:\n", x_train[0])

## 5. 검증 세트 준비

In [ ]:
# 훈련 데이터 중 10,000개를 검증 세트로 분리
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

print(f"훈련 데이터 수: {len(partial_x_train)}")
print(f"검증 데이터 수: {len(x_val)}")

## 6. 모델 구축 및 컴파일

In [ ]:
model = models.Sequential([
    layers.Dense(16, activation="relu", input_shape=(10000,)),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid") # 이진 분류이므로 sigmoid 활성화 함수 사용
])

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy", # 이진 분류를 위한 손실 함수
              metrics=["accuracy"])

model.summary()

## 7. 모델 훈련

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

## 8. 훈련 결과 시각화

In [ ]:
history_dict = history.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
acc = history_dict["accuracy"]
val_acc = history_dict["val_accuracy"]
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(14, 5))

# 손실 그래프
plt.subplot(1, 2, 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

# 정확도 그래프
plt.subplot(1, 2, 2)
plt.plot(epochs, acc, "bo", label="Training acc")
plt.plot(epochs, val_acc, "b", label="Validation acc")
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.show()

## 9. 모델 평가 및 예측

In [ ]:
print("--- 테스트 데이터셋 평가 ---")
results = model.evaluate(x_test, y_test)
print(f"테스트 손실: {results[0]:.4f}")
print(f"테스트 정확도: {results[1]:.4f}")

print("\n--- 예측 결과 샘플 ---")
predictions = model.predict(x_test)
for i in range(10):
    pred_label = "긍정" if predictions[i][0] > 0.5 else "부정"
    true_label = "긍정" if y_test[i] == 1 else "부정"
    print(f"리뷰 {i+1}: 예측={pred_label} (값: {predictions[i][0]:.3f}), 실제={true_label}")